<a href="https://colab.research.google.com/github/jonggu12/yorehalre_project/blob/main/llama2_%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D%EC%9A%A9_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 구글드라이브 연계

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


★ 설정부분: 본인의 학습데이터가 저장되어 있는 폴더경로를 설정하세요


예: "/content/gdrive/MyDrive/Colab Notebooks/llama2_custom/dataset/"


In [ ]:
dataPath = "/content/gdrive/MyDrive/colab Notebooks/senior_hccare_dataset/"

### 라이브러리 설치

In [ ]:
# datasets: hugging-face의 인공지능 커뮤니티의 데이터를 불러오고 관리하는 라이브러리
# 기존 라이브러리가 2월19일 기준 업데이트 되면서 pyarrow 라이브러리와 호환성 문제 발생 띠리사 호환라이브러리 정의
# jsonlines: json 포맷의 데이터를 핸들링하는 라이브러리
!pip install datasets==2.16.1 jsonlines==4.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00


### huggign face 접근을 위한 로그인

In [ ]:
# hugging face 로그인: hugging face -> 우측상단의 계정선택 -> settings -> access token
import huggingface_hub
huggingface_hub.login()

### 설치한 라이브러리 선언

In [ ]:
# 행과열을 핸들링하는 라이브러리
import pandas as pd
# json 포맷 데이터를 핸들링하는 라이브러리
import json
import jsonlines
# hugging face 데이터 관리 라이브러리
from datasets import Dataset

1. 학습 데이터 불러오기 및 변환

★ 설정부분: 본인의 학습데이터 파일명을 설정하세요


예: "/content/gdrive/MyDrive/Colab Notebooks/llama2_custom/dataset/" 경로 내 indata_kor.csv 파일이면 indata_kor.csv

jsonFileName은 그냥 동일이름의 .jsonl이면 됨


In [ ]:
# 데이터 경로 설정 및 불러오기
datasetName = "senior_healthcare_dataset.csv"
jsonFileName = "senior_healthcare_dataset.jsonl"

In [ ]:
def csv_to_json(csv_file_path, json_file_path):
    # CSV 파일을 DataFrame으로 읽기★ (한글인 경우 encoding="ms949") 영문인경우 해당 내용 삭제 ★
    df = pd.read_csv(csv_file_path)

    # JSON 파일로 저장
    with open(json_file_path, 'w', encoding='utf-8') as json_file:
        # 각 행을 JSON으로 변환하여 바로 파일에 쓰기
        for index, row in df.iterrows():
            data = {'inputs': row['inputs'], 'response': row['response']}
            json.dump(data, json_file, ensure_ascii=False)
            json_file.write('\n')  # 각 행마다 줄바꿈

# CSV 파일 경로와 JSON 파일 경로 설정
csv_file_path = dataPath + datasetName
json_file_path = dataPath + jsonFileName

# 함수 호출
csv_to_json(csv_file_path, json_file_path)

2. 파인튜닝용 포맷으로 변환

In [ ]:
indataset = []
with jsonlines.open(json_file_path) as f:
    for line in f.iter():
      indataset.append(f'<s>[INST] {line["inputs"]} [/INST] {line["response"]} </s>')
    #   indataset.append(f'<s>### Instruction: \n{line["inputs"]} \n\n### Response: \n{line["response"]}</s>')

# 데이터셋 확인
print('데이터셋 확인')
print(indataset[:5])

# 데이터셋 생성 및 저장
indataset = Dataset.from_dict({"text": indataset})
indataset.save_to_disk(dataPath)

# 데이터셋 info 확인
print('데이터셋 info 확인')
print(indataset)

데이터셋 확인
['<s>[INST] 기초대사량의 변화에 대한 설명해주세요. [/INST] 오랫동안 기아상태로 체중을 감소시키면 기초대사가 10% 이상 감소한다.개인의 근육량에 영향을 받는다. 체온이 상승함에 따라 증가된다.\r 수면 시 기초대사량이 10% 감소한다.\r\n </s>', '<s>[INST] 휴식대사량에 대한 설명해주세요. [/INST] 1. 식후 2~3시간 후에 아무런 근육활동 없이 편안한 자세에서 측정한다. 2. 1 일 에너지 소모량의 60~75%를 차지한다. ⑧ 개인의 근육량에 영향을 받는다 4. 식이성 발열효과의 영향을 어느 정도 받는다. </s>', '<s>[INST] 「2020 한국인 영양소 섭취기준」중 상한섭취량이 설정되어 있는 것은? [/INST]  2020 한국인 영양소 섭취기준은 에너지 및 다량영양소 12종, 비타민 13종, 무기질 15종의\r\n총 40종 영양소에 대해 설정되었다.\r\n2020 한국인 영양소 섭취기준 제  개정에서는 탄수화물에 대한 펑균필요량과\r\n지방산(리놀레산, 알파-리놀렌산, DHA+EPA)에 대한 충분섭취량이 새롭게 제정되었으며 ,\r\n단백질에 대한 평균필요량과 권장섭취량이 개정되었다.\r\n 상한섭취량이 설정된 영양소\r\n- 지용성 비타민 : 비타민 A, 비탁민 D, 비타민 E\r\n- 수용성 비타민 : 비타민 C, 니아신, 비타민 Be. 엽산\r\n- 무기질 : 칼슘, 인, 마그네슘, 철, 아연, 구리, 불소, 망간, 요오드, 셀레늄, 몰리브덴 </s>', '<s>[INST] 「2020 한국인 영양소 섭취기준」 중 인구집단의 97~98%에 해당하는 사람들의 영양소 필요량을 섭취량으로 나타 낸 것으로 평균필요량에 표준편차 또는 변이계수의 2배를 더하여 산출한 값은? [/INST] 안전하고 충분한 영양을 확보하는 기준치(평균필요량, 권장섭취량, 충분섭취량, 상한섭취량)와 식사와 관련된 만성 질환 위험감소를 고려한 기준치( 에너지 적정비율, 만성질환위험 감소섭취량)를 제시한다. </s>', '<s>[

Saving the dataset (0/1 shards):   0%|          | 0/334 [00:00<?, ? examples/s]

데이터셋 info 확인
Dataset({
    features: ['text'],
    num_rows: 334
})


3. huggingface 에 데이터 업로드 (★ 변경 포인트!)

In [ ]:
indataset.push_to_hub("jiapn/senior_healthcare")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jiapn/senior_healthcare/commit/3f5f92f6719eaaf4f965e99ba8c332aeff74a783', commit_message='Upload dataset', commit_description='', oid='3f5f92f6719eaaf4f965e99ba8c332aeff74a783', pr_url=None, pr_revision=None, pr_num=None)